# look vec estimator

<br>

## Baseline 모델 
#### - mobilenet_v2_1.0_128

<br>

## Note
#### - look-vec은 정규화된 벡터이므로 3개 중 2개만 추정해도 됨
#### - in head-pose는 roll 성분이 0이지만 확장을 위해 3개 다 추가


<br>

## Table
|exp|type|filter|warp|aug|rot|
|---|---|---|---|---|---|
|exp.1|regression|x|o|o|x|

# imports

In [1]:
import sys
import keras
import keras.backend as K

from keras.models import Model
from keras.layers import Dense, Flatten
from keras.engine.input_layer import Input
from keras.layers.merge import Concatenate, Add
from keras.losses import sparse_categorical_crossentropy
from keras.metrics import sparse_top_k_categorical_accuracy
from keras.applications.mobilenet_v2 import MobileNetV2

Using TensorFlow backend.


In [2]:
sys.path.append("../../../")

%run ../../../ds/unity/npz/gen.py
%run ../../../ac/visualizer/plotter.py
%run ../../../ai/libs/keras/callbacks/history.py
%run ../../../ai/libs/keras/callbacks/checkpoint.py
%run ../../../ai/libs/keras/callbacks/stopper.py

# resource paths

In [3]:
npz_path = "/home/chy/archive-data/processed/unity-class-shuffled-npz"
chk_path = "/home/chy/archive-model/incubator/lookvec/lv-{epoch:02d}-{val_loss:.7f}.hdf5"

# model

In [4]:
INPUT_SHAPE=(56,112,1)

In [5]:
mb2 = MobileNetV2(input_shape=INPUT_SHAPE, 
                    alpha=1.0, 
                    depth_multiplier=1, 
                    include_top=False, 
                    weights=None, 
                    input_tensor=None, 
                    pooling=None)
input_hp = Input(shape=(3,))
last = mb2.output

x = Flatten()(last)
x = Dense(512)(x)
x = Concatenate(axis=1)([input_hp, x])
preds = Dense(2, activation='linear')(x)

lvm = Model(inputs=[mb2.input, input_hp], outputs=preds)
lvm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 56, 112, 1)   0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 57, 113, 1)   0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 28, 56, 32)   288         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 28, 56, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

# callback

In [6]:
history = TimeHistory()
checkpoint = model_checkpoint(chk_path)
stopper = EarlyStopping(monitor='val_loss', 
                        min_delta=0, 
                        patience=5, 
                        verbose=0, 
                        mode='auto', 
                        baseline=None, 
                        restore_best_weights=True)

callbacks = [history, checkpoint, stopper]

# compile

In [7]:
lvm.compile(loss='mean_squared_error', 
            optimizer='adam', 
            metrics=['mae', 'acc'])

# hyper-params

In [8]:
EXP_CODE = "pilot"
NUM_EPOCH = 20
BATCH_SIZE = 250

# model-meta back up

In [9]:
model_json = lvm.to_json()
with open("./lvm-{}.json".format(EXP_CODE), "w") as json_file : 
    json_file.write(model_json)

# train

In [13]:
gen_train = UnityEyePoseGenerator(npz_base_path=npz_path, 
                                  batch_size=BATCH_SIZE, 
                                  purpose=Purpose.TRAIN,
                                  use_postprocess=False,
                                  use_aug=True)

gen_valid = UnityEyePoseGenerator(npz_base_path=npz_path, 
                                  batch_size=BATCH_SIZE, 
                                  purpose=Purpose.VALID,
                                  use_postprocess=False,
                                  use_aug=True)

*** meta verify complete [unity 1681 class shuffled] ***
*** meta verify complete [unity 1681 class shuffled] ***


In [14]:
lvm.fit_generator(generator=gen_train,
                  validation_data=gen_valid,
                  callbacks=callbacks, 
                  workers=16, 
                  use_multiprocessing=True,
                  shuffle=True)

Epoch 1/1
23525/23525 [==============================] - 23454s 997ms/step - loss: 0.0812 - mean_absolute_error: 0.1425 - acc: 0.8430 - val_loss: 0.0081 - val_mean_absolute_error: 0.0663 - val_acc: 0.9305


OSError: Unable to create file (unable to open file: name = '/home/chy/archive-model/incubator/lookvec/lv-01-0.0080608.hdf5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)